### Summary of Categorical Var

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust

import seaborn as sns
import matplotlib.pylab as plt

import matplotlib.font_manager as fm
import matplotlib

fm.get_fontconfig_fonts()
font_location = "c:/windows/Fonts/malgun.ttf"
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

Loan Data
- P2P 방식의 대출 사업을 선도하고 있는 Lending Club에서 제공하는 데이터
- 대출등급(grade) : A-G
- 대출상환 상태(status) : Fully paid(전액 상환), Current(진행 중), Late(연체), ChargedOff(삭제: 대출 잔여금 회수 불능)

In [4]:
data_path = "./data/"

lc_loans = pd.read_csv(data_path+'lc_loans.csv')
lc_loans.head()

,status,grade
0,Fully Paid,B
1,Charged Off,C
2,Fully Paid,C
3,Fully Paid,C
4,Current,B


In [6]:
lc_loans.shape, lc_loans.dtypes

((450961, 2),
 status    object
 grade     object
 dtype: object)

In [7]:
lc_loans.describe()
# top: mode (최빈값)

,status,grade
count,450961,450961
unique,4,7
top,Current,B
freq,321185,132370


In [8]:
lc_loans.status.nunique()

4

In [9]:
lc_loans.status.value_counts()

Current        321185
Fully Paid      97316
Charged Off     22671
Late             9789
Name: status, dtype: int64

In [10]:
lc_loans.grade.nunique()

7

In [11]:
lc_loans.grade.value_counts()

B    132370
C    120875
D     74277
A     72490
E     34804
F     12904
G      3241
Name: grade, dtype: int64

In [12]:
lc_loans.isna().sum()

status    0
grade     0
dtype: int64

In [13]:
lc_loans.status= lc_loans.status.astype('category')
lc_loans.grade = lc_loans.grade.astype('category')

lc_loans.dtypes

status    category
grade     category
dtype: object

In [14]:
lc_loans.status.value_counts(sort=False)

Charged Off     22671
Current        321185
Fully Paid      97316
Late             9789
Name: status, dtype: int64

In [15]:
lc_loans.grade.value_counts(sort=False)

A     72490
B    132370
C    120875
D     74277
E     34804
F     12904
G      3241
Name: grade, dtype: int64

In [16]:
# 범주 순서 변경
lc_loans.status = lc_loans.status.cat.reorder_categories(['Fully Paid', 'Current','Late', 'Charged Off' ])
lc_loans.status.value_counts(sort=False)

Fully Paid      97316
Current        321185
Late             9789
Charged Off     22671
Name: status, dtype: int64

#### 막대도표 (Bar plot)